In [1]:
import pandas as pd
import numpy as np
from pyjet import cluster,DTYPE_PTEPM

# Pre processing

In [2]:
path = path = '/anomalyvol/data/events_LHCO2020_backgroundMC_Pythia.h5'

In [3]:
df = pd.read_hdf(path,stop=1000) # just read first 1000 events

In [4]:
all_events = df.values

In [5]:
rows = all_events.shape[0]
cols = all_events.shape[1]
data = []

In [6]:
for i in range(rows):
    pseudojets_input = np.zeros(len([x for x in all_events[i][::3] if x > 0]), dtype=DTYPE_PTEPM)
    for j in range(cols // 3):
        if (all_events[i][j*3]>0):
            pseudojets_input[j]['pT'] = all_events[i][j*3]
            pseudojets_input[j]['eta'] = all_events[i][j*3+1]
            pseudojets_input[j]['phi'] = all_events[i][j*3+2]
        pass
    sequence = cluster(pseudojets_input, R=1.0, p=-1)
    jets = sequence.inclusive_jets()
    for k in range(len(jets)):
        jet = []
        jet.append(jets[k].pt)
        jet.append(jets[k].eta)
        jet.append(jets[k].phi)
        jet.append(jets[k].mass)
        data.append(jet)

In [7]:
data = np.array(data)
data.shape

(14795, 4)

# VAE Model

In [8]:
import keras
from keras.layers import Lambda, Input, Dense, Flatten, Reshape
from keras.models import Model, Sequential
from keras import metrics
from keras import backend as K

Using TensorFlow backend.


In [9]:
def sampling(args):
    """
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [10]:
inter_dim = 32
final_dim = 8
latent_dim = 2
input_dim = 4

In [11]:
# encoder
x = Input(shape=(input_dim,))
#x_flat = Flatten()(x)
h1 = Dense(inter_dim, activation='relu')(x)
h2 = Dense(final_dim, activation='relu')(h1)
z_mean = Dense(latent_dim)(h2)
z_log_sigma = Dense(latent_dim)(h2)

In [12]:
# random sampling
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

encoder = Model(inputs = x, outputs = z)

In [13]:
# decoder
decoder_h2 = Dense(final_dim, activation='relu')
decoder_h1 = Dense(inter_dim, activation='relu')
decoder_mean = Dense(input_dim, activation='sigmoid')

h2_decoded = decoder_h2(z)
h1_decoded = decoder_h1(h2_decoded)
x_decoded_mean = decoder_mean(h1_decoded)
# x_decoded = Reshape(input_shape)(x_decoded_mean)

In [14]:
vae = Model(inputs = x, outputs = x_decoded_mean, name = 'vae')

In [15]:
def vae_loss(x, y):
    xent_loss = metrics.binary_crossentropy(x, y)
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    return xent_loss + kl_loss

vae.compile(optimizer='adam', loss=vae_loss)

In [16]:
vae.summary()

Model: "vae"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 32)           160         input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 8)            264         dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 2)            18          dense_2[0][0]                    
________________________________________________________________________________________________

# Train

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
x_train = data[0:int(len(data) * 0.8)]
x_val = data[int(len(data) * 0.8):]
batch_size = 100
epochs = 150

In [19]:
x_train.shape

(11836, 4)

In [20]:
hist = vae.fit(x_train, x_train,
               shuffle=True,
               epochs=epochs,
               batch_size=batch_size,
               validation_data=(x_val, x_val))

Train on 11836 samples, validate on 2959 samples
Epoch 1/150
11836/11836 [==============================] - 1s 108us/step - loss: nan - val_loss: nan
Epoch 2/150
11836/11836 [==============================] - 1s 49us/step - loss: nan - val_loss: nan
Epoch 3/150
11836/11836 [==============================] - 1s 50us/step - loss: nan - val_loss: nan
Epoch 4/150
11836/11836 [==============================] - 1s 50us/step - loss: nan - val_loss: nan
Epoch 5/150
11836/11836 [==============================] - 1s 51us/step - loss: nan - val_loss: nan
Epoch 6/150
11836/11836 [==============================] - 1s 49us/step - loss: nan - val_loss: nan
Epoch 7/150
11836/11836 [==============================] - 1s 49us/step - loss: nan - val_loss: nan
Epoch 8/150
11836/11836 [==============================] - 1s 48us/step - loss: nan - val_loss: nan
Epoch 9/150
11836/11836 [==============================] - 1s 49us/step - loss: nan - val_loss: nan
Epoch 10/150
11836/11836 [========================

KeyboardInterrupt: 